In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline`

### Data Alignment Problem for Time Series

Pandas automatically align dataframes for arithmetic operations

Pandas functions ignore NaNs - treat them as non-existant (len=0)

### Can choose to only work with the shared variables using pd.align
Returns a tuple:
    
    prices.align(volume, join = 'inner')

In [ ]:
prices = pd.read_csv('ch11/prices.csv')
prices = prices.set_index('Unnamed: 0')
prices.index.name = 'date'

volume = pd.read_csv('ch11/volumes.csv')
volume = volume.set_index('Unnamed: 0')
volume.index.name = 'date'

prices = prices[['AAPL','JNJ','SPX','XOM']]
volume = volume[['AAPL','JNJ','XOM']]
prices = prices['2011-09-06':'2011-09-14']
volume = volume['2011-09-06':'2011-09-12']

print prices
print volume

In [ ]:
# Arithmetic operations are automatically aligned
prices * volume

In [ ]:
# Pandas functions also aggregate over NaNs w/o error
# Can easily calculate volume-weighted average price across the time range
vwap = (prices * volume).sum() / volume.sum()
vwap

### Can choose to only work with the shared variables using pd.align
<b>Returns a tuple

In [43]:
prices.align(volume, join = 'inner')

(                       AAPL    JNJ    XOM
 date                                     
 2011-09-06 00:00:00  379.74  64.64  71.15
 2011-09-07 00:00:00  383.93  65.43  73.65
 2011-09-08 00:00:00  384.14  64.95  72.82
 2011-09-09 00:00:00  377.48  63.64  71.01,
                            AAPL         JNJ         XOM
 date                                                   
 2011-09-06 00:00:00  18173500.0  15848300.0  25416300.0
 2011-09-07 00:00:00  12492000.0  10759700.0  23108400.0
 2011-09-08 00:00:00  14839800.0  15551500.0  22434800.0
 2011-09-09 00:00:00  20171900.0  17008200.0  27969100.0)